In [ ]:
from megs import model
from megs.data import image_modules

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from megs import *

In [ ]:
import megs

In [ ]:
from megs.data import galaxy

In [ ]:
data = model.DataLoader("/export/home/ucakir/MEGS/MEGS/data/galaxy_data.hdf5", m_min = 8)

In [ ]:
norm = image_modules.norm # Normalization function
lower = 0.25
upper = 1.0
norm_function_args = {"Masses": {"takelog": True, "plusone": True,"lower": lower, "upper": upper},
                     "GFM_Metallicity": {"takelog": True, "plusone": True, "lower": 0.1, "upper": upper},
                     "GFM_StellarFormationTime": {"takelog": True, "plusone": True, "lower": 0.1, "upper": upper},
                    
}

In [ ]:
plt.hist(np.log10(masses)) 

In [ ]:
morphmodel = model.mPCA(data, norm_function = norm, norm_function_kwargs=norm_function_args)

In [ ]:
morphmodel.fit(n_components=60)

In [ ]:
def show_pca_results(self):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].plot(self.pca.explained_variance_ratio_)
    ax[0].set_xlabel("Component")
    ax[0].set_ylabel("Explained variance ratio")
    ax[1].plot(np.cumsum(self.pca.explained_variance_ratio_))
    ax[1].set_xlabel("Component")
    ax[1].set_ylabel("Cumulative explained variance ratio")
    plt.show()
    
    
    field_length = len(self.data._image_fields[self.particle_type]) 
    
    #Loop over different image fields
    for index,field in enumerate(self.data._image_fields[self.particle_type]):

        

        rows = int(np.ceil(np.sqrt(self.pca.n_components)))
        cols = int(np.ceil(self.pca.n_components/rows))
        fig, ax = plt.subplots(rows, cols, figsize=(cols*3, rows*3))
        for i in range(rows):
            for j in range(cols):
                if i*cols+j < self.pca.n_components:
                    ax[i, j].imshow(self.eigengalaxies[i*cols+j][index])
                    ax[i, j].set_title(f"Component {i*cols+j}")
                    ax[i, j].axis("off")
        fig.suptitle(f"Eigengalaxies:{field}")
        plt.show()
        
        
    
  
    # Plot the mean galaxy
    
    fig, ax = plt.subplots(1,field_length, figsize = (field_length*3, 3))
    for index,field in enumerate(self.data._image_fields[self.particle_type]):
        ax[index].imshow(self.pca.mean_.reshape(field_length, *self._IMG_SHAPE)[index])
        ax[index].set_title(f"{field}")
        ax[index].axis("off")
    fig.suptitle("Mean Galaxy")
    plt.show() 
            
            
    #Calculate residue of random galaxy
    randomind = np.random.randint(0, self.datamatrix.shape[0])
    inverse_images = self.inverse_transformed_datamatrix[randomind].reshape(field_length, *self._IMG_SHAPE)
    for index, field in enumerate(self.data._image_fields[self.particle_type]):
        fig, ax = plt.subplots(1, 3, figsize=(6, 3))
        
        original = self.datamatrix[randomind].reshape(field_length, *self._IMG_SHAPE)[index]
        
        ax[0].imshow(original)
        ax[0].set_title(f"Original galaxy")
        ax[0].axis("off")
        ax[1].imshow(inverse_images[index])
        ax[1].set_title(f"Reconstructed galaxy")
        ax[1].axis("off")
        
        # Calculate residue
        residue = np.abs(original - inverse_images[index])
        ax[2].imshow(residue)
        ax[2].set_title(f"Residue")
        ax[2].axis("off")
        
        fig.suptitle(f"Comparison of {field} field")
        plt.show()

In [ ]:
show_pca_results(morphmodel)

In [ ]:
mass = data.get_image(field="Masses", particle_type="stars", index = 20)
metal = data.get_image(field="GFM_Metallicity", particle_type="stars", index = 20)
age = data.get_image(field= "GFM_StellarFormationTime", particle_type="stars", index = 20)


#plot all three images
fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(norm(mass,**norm_function_args["Masses"]))
ax[0].set_title("Mass")
ax[0].axis("off")
ax[1].imshow(norm(metal, **norm_function_args["GFM_Metallicity"]))
ax[1].set_title("Metallicity")
ax[1].axis("off")
ax[2].imshow(norm(age, **norm_function_args["GFM_StellarFormationTime"]))
ax[2].set_title("Age")
ax[2].axis("off")
plt.show()


In [ ]:
compare(morphmodel,20)

In [ ]:
eigen =morphmodel.eigengalaxies

In [ ]:
EIGENORDER = morphmodel.data._image_fields["stars"]
EIGENORDER

In [ ]:
mass_eigen = eigen[:,EIGENORDER.index("Masses")]
metal_eigen = eigen[:,EIGENORDER.index("GFM_Metallicity")]
age_eigen = eigen[:,EIGENORDER.index("GFM_StellarFormationTime")]

In [5]:
string = "dim2"
dim = int(string[3])

In [6]:
string[3]

'2'

In [3]:
int(string)

ValueError: invalid literal for int() with base 10: 'dim3'

In [2]:
dim

3

In [ ]:
mean_img = morphmodel.pca.mean_.reshape(3, 64, 64)[EIGENORDER.index("Masses")]
mean_metal_img = morphmodel.pca.mean_.reshape(3, 64, 64)[EIGENORDER.index("GFM_Metallicity")]
mean_age_img = morphmodel.pca.mean_.reshape(3, 64, 64)[EIGENORDER.index("GFM_StellarFormationTime")]

In [ ]:
mass_img_normed = norm(mass, **norm_function_args["Masses"])

In [ ]:
morphmodel.pca.n_components

In [ ]:
n_components = morphmodel.pca.n_components

mass_img_centered = mass_img_normed - mean_img
mass_img_centered_reshaped = mass_img_centered.reshape(64*64)
mass_eigen_reshaped = mass_eigen.reshape(n_components,64*64)


weights = np.dot(mass_img_centered_reshaped, mass_eigen_reshaped.T) 

In [ ]:
weights.shape

In [ ]:
age_img = norm(age, **norm_function_args["GFM_StellarFormationTime"]) - mean_age_img

weights = np.dot(age_img.flatten(), age_eigen.reshape(n_components,64*64).T)

In [ ]:
mass_eigen.shape

In [ ]:
galaxy = morphmodel.datamatrix[20]

In [ ]:
galaxy.shape

In [ ]:
all_eigen = eigen.reshape(60,-1)
all_means = morphmodel.pca.mean_

In [ ]:
weights = np.dot(galaxy[:64*64] - all_means[:64*64], all_eigen[:,:64*64].T)

inverse = np.dot(weights, all_eigen) + all_means
#images = inverse.reshape(3, 64, 64)

images = inverse.reshape(3,64,64)


fig,ax = plt.subplots(1,3, figsize = (10,5))
ax[0].imshow(images[0])
ax[1].imshow(images[1])
ax[2].imshow(images[2])

In [ ]:
all_eigen.shape

In [ ]:
reconstruction_mass_img = np.dot(weights, mass_eigen.reshape(n_components,64*64)) + mean_img.flatten()
reconstruction_metal_img = np.dot(weights, metal_eigen.reshape(n_components,64*64)) + mean_metal_img.flatten()
reconstruction_age_img = np.dot(weights, age_eigen.reshape(n_components,64*64)) + mean_age_img.flatten()


# show the reconstructed images

fig, ax = plt.subplots(1, 3, figsize=(10, 5))

ax[0].imshow(reconstruction_mass_img.reshape(64,64))
ax[0].set_title("Mass")
ax[0].axis("off")

ax[1].imshow(reconstruction_metal_img.reshape(64,64))
ax[1].set_title("Metal")
ax[1].axis("off")


ax[2].imshow(reconstruction_age_img.reshape(64,64))
ax[2].set_title("Age")
ax[2].axis("off")



In [ ]:
reconstruction_mass_img = np.dot(weights, mass_eigen.reshape(n_components,64*64)) + mean_img.flatten()
reconstruction_metal_img = np.dot(weights, metal_eigen.reshape(n_components,64*64)) + mean_metal_img.flatten()
reconstruction_age_img = np.dot(weights, age_eigen.reshape(n_components,64*64)) + mean_age_img.flatten()


# show the reconstructed images

fig, ax = plt.subplots(1, 3, figsize=(10, 5))

ax[0].imshow(reconstruction_mass_img.reshape(64,64))
ax[0].set_title("Mass")
ax[0].axis("off")

ax[1].imshow(reconstruction_metal_img.reshape(64,64))
ax[1].set_title("Metal")
ax[1].axis("off")


ax[2].imshow(reconstruction_age_img.reshape(64,64))
ax[2].set_title("Age")
ax[2].axis("off")



In [ ]:
def compare(self, index = None):
    field_length = len(self.data._image_fields[self.particle_type])
     #Calculate residue of random galaxy
    if index is None:
        randomind = np.random.randint(0, self.datamatrix.shape[0])
    else:
        randomind = index
    inverse_images = self.inverse_transformed_datamatrix[randomind].reshape(field_length, *self._IMG_SHAPE)
    for index, field in enumerate(self.data._image_fields[self.particle_type]):
        fig, ax = plt.subplots(1, 3, figsize=(6, 3))
        
        original = self.datamatrix[randomind].reshape(field_length, *self._IMG_SHAPE)[index]
        
        ax[0].imshow(original)
        ax[0].set_title(f"Original galaxy")
        ax[0].axis("off")
        ax[1].imshow(inverse_images[index])
        ax[1].set_title(f"Reconstructed galaxy")
        ax[1].axis("off")
        
        # Calculate residue
        residue = np.abs(original - inverse_images[index])
        #Divide with original where original != 0
        residue[original != 0] = residue[original != 0] / original[original != 0]
        print(residue.max(), residue.min())
        ax[2].imshow(residue, vmin=0, vmax=1)
        ax[2].set_title(f"Residue")
        ax[2].axis("off")
        
        fig.suptitle(f"Comparison of {field} field")
        plt.show()

In [ ]:
compare(morphmodel,20)

In [ ]:
masses = data.get_attribute("mass")
ids = data.get_attribute("halo_id")

In [ ]:
likelihood = morphmodel.pca.score_samples(morphmodel.datamatrix)

In [ ]:
outlier = np.argsort(likelihood)

In [ ]:
ids[outlier[151]]

In [ ]:
plt.hist(np.log10(masses[outlier[:1000]]))

In [ ]:
compare(morphmodel, outlier[12])

In [ ]:
np.log10(masses).min()